# Detection

### Importing Necessary Libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import mediapipe as mp
from IPython.utils import io
import math
from scipy.spatial import distance

### Classes

In [2]:
CATEGORIES = ['WithoutMask', 'WithMask']

### Creating MediaPipe FaceDetection Object

In [3]:
mp_face_detection = mp.solutions.face_detection
mp_pose = mp.solutions.pose

We will be using this module for the detection of faces. 

### Loading the Model

In [4]:
MaskModel=tf.keras.models.load_model('MaskDetectionCORE.h5')

This model was trained on thousands of images to detect if a face is Masked or Unmasked

### Initializing YOLO Model

In [5]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

### Defining Classes of YOLO

In [6]:
classes = []
with open('coco.names', 'r') as f:
    classes = f.read().splitlines()

### Initializing MP objects

In [7]:
face_detector = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.3)

### Detection from Live Video [ Webcam ]

In [8]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    selfie = cv2.flip(frame, 1)
    mirror = cv2.flip(frame, 1)
    height, width = mirror.shape[:2]
    mirror = cv2.cvtColor(mirror, cv2.COLOR_BGR2RGB)
    results = face_detector.process(mirror)
    mirror = cv2.cvtColor(mirror, cv2.COLOR_RGB2BGR)
    if results.detections:
        for face in results.detections:
            bounding_box = face.location_data.relative_bounding_box
            x = int(bounding_box.xmin * width)
            y = int(bounding_box.ymin * height)
            w = int(bounding_box.width * width)
            h = int(bounding_box.height * height)
            img = mirror[y:y+h,x:x+w]
            try:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            except:
                break
            threed = np.repeat(gray[...,np.newaxis], 3, -1)
            preprocessed = preprocess_input(threed)
            resized = cv2.resize(preprocessed, (224,224))
            with io.capture_output() as captured:
                predictions = MaskModel.predict(np.expand_dims(resized, axis=0))
            accuracy = str(round(predictions.max(), 2))
            predicted = CATEGORIES[predictions.argmax()]
            if predicted == 'WithMask':
                cv2.rectangle(selfie, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cv2.putText(selfie, 'MASKED', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            elif predicted == 'WithoutMask':
                cv2.rectangle(selfie, (x, y), (x+w, y+h), (0, 0, 255), 3)
                cv2.putText(selfie, 'UNMASKED', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    blob = cv2.dnn.blobFromImage(mirror, 1/255, (width, height), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layer_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layer_names)
    boxes = []
    confidences = []
    for detections in layerOutputs:
        for detection in detections:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id] 
            if confidence > 0.8 and class_id == 0:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.5)
    if len(indexes) > 0:
        centroids = []
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            c = (int((x+x+w)/2), int((y+y+h)/2))
            centroids.append(c)
            label = 'PERSON'
            uniqueID = str(np.where(indexes.flatten() == i)[0])[1:-1]
            confidence = str(round(confidences[i], 2))
            cv2.rectangle(selfie, (x,y), (x+w, y+h), (255, 0, 0), 4)
            cv2.putText(selfie, label+" "+uniqueID, (x, y+30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        if len(centroids) > 1:
            distmat = distance.pdist(centroids)
            mindist = distmat[distmat.argmin()]
            if mindist < 300:
                cv2.putText(selfie, 'MAINTAIN SOCIAL DISTANCE', (0,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(selfie, 'NO PERSONS DETECTED', (width-400, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('Webcam', selfie)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()  

 ### Process given video & Save it as output

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'xvid')
out = cv2.VideoWriter('Output.mp4', fourcc, 14.0, (640, 480))
cap = cv2.VideoCapture('VideoForDetection.mp4')
while cap.isOpened():
    _, frame = cap.read()
    if _ != True:
        break
    selfie = cv2.flip(frame, 1)
    mirror = cv2.flip(frame, 1)
    height, width = mirror.shape[:2]
    mirror = cv2.cvtColor(mirror, cv2.COLOR_BGR2RGB)
    results = face_detector.process(mirror)
    mirror = cv2.cvtColor(mirror, cv2.COLOR_RGB2BGR)
    if results.detections:
        for face in results.detections:
            bounding_box = face.location_data.relative_bounding_box
            x = int(bounding_box.xmin * width)
            y = int(bounding_box.ymin * height)
            w = int(bounding_box.width * width)
            h = int(bounding_box.height * height)
            img = mirror[y:y+h,x:x+w]
            try:
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            except:
                break
            threed = np.repeat(gray[...,np.newaxis], 3, -1)
            preprocessed = preprocess_input(threed)
            resized = cv2.resize(preprocessed, (224,224))
            with io.capture_output() as captured:
                predictions = MaskModel.predict(np.expand_dims(resized, axis=0))
            accuracy = str(round(predictions.max(), 2))
            predicted = CATEGORIES[predictions.argmax()]
            if predicted == 'WithMask':
                cv2.rectangle(selfie, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cv2.putText(selfie, 'MASKED', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            elif predicted == 'WithoutMask':
                cv2.rectangle(selfie, (x, y), (x+w, y+h), (0, 0, 255), 3)
                cv2.putText(selfie, 'UNMASKED', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    blob = cv2.dnn.blobFromImage(mirror, 1/255, (640, 480), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layer_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layer_names)
    boxes = []
    confidences = []
    for detections in layerOutputs:
        for detection in detections:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id] 
            if confidence > 0.8 and class_id == 0:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.5)
    if len(indexes) > 0:
        centroids = []
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            c = (int((x+x+w)/2), int((y+y+h)/2))
            centroids.append(c)
            label = 'PERSON'
            uniqueID = str(np.where(indexes.flatten() == i)[0])[1:-1]
            confidence = str(round(confidences[i], 2))
            cv2.rectangle(selfie, (x,y), (x+w, y+h), (255, 0, 0), 4)
            cv2.putText(selfie, label+" "+uniqueID, (x, y+30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
        if len(centroids) > 1:
            distmat = distance.pdist(centroids)
            mindist = distmat[distmat.argmin()]
            if mindist < 325:
                cv2.putText(selfie, 'ALERT SOCIAL DISTANCE', (40,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
    else:
        cv2.putText(selfie, 'NO PERSONS DETECTED', (width-400, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    out.write(selfie)
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
out.release()
cap.release()
cv2.destroyAllWindows()

### Capture WebCam Video For Processing

In [10]:
fourcc = cv2.VideoWriter_fourcc(*'xvid')
out = cv2.VideoWriter('VideoForDetection.mp4', fourcc, 20.0, (640, 480))
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, frame = cap.read()
    selfie = cv2.flip(frame, 1)
    if _:
        cv2.imshow('WEBCAM', selfie)
        out.write(selfie)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            print('Quitting')
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
out.release()

Quitting
